# Objekterkennung IntSys

In diesem Projekt sollen Objekte auf einem Drehteller mithilfe eines Ultraschall Sensors (HCSR04) erkannt werden.

Die gemessenen Daten befinden sich im Unterordner "messdaten"

### Primär orientieren wir uns an der digits-erkennen Übung, und benutzen sklearn

In [2]:
import matplotlib.pyplot as plt
from sklearn import datasets, metrics, svm
from sklearn.model_selection import train_test_split
import pandas as pd

# Datenformat

Jede log-Datei in den Messdaten besteht aus 10 Umdrehungen.
Zum modellieren, wird die raw data 50/50 geteilt, und anschliessend vorbereitet

Messdateninfos:
Die Messdaten haben jeweils 61 Messpunkte/Umdrehung

In [31]:
directory = "messdaten/"

file = "logfile_deo_dose_53mm.txt"

df = pd.read_csv(directory + file, header=None)

In [36]:
def transform_data(df):
    """
    input 305 measurement points, > 5 turns * 61 measurements
    für jeden messpunkt einer drehung die nächsten 61 punkte anhängen
         > ACHTUNG!: nur wenn es 61 oder mehr messpunkte bis zum ende des input df gibt!

         > d.h. es gibt 305 - 61 = 244 output zeilen im df
    """
    # transpose from (n, 1) -> (1, n)
    df_transposed = df.T
    output_data = []

    for i in range(244): # 244 because 305 - 61 (letzte Umdrehung wird rausgenommen, damit jeder neue teil eine komplette umdrehung hat)
        one_turn = df_transposed.iloc[:, i:i+61]
        print(one_turn)
        # output_data.append(one_turn)
     
    output_df = pd.DataFrame(output_data)

    return output_df

# def label_data(df, label):
    # """ergänzt dataframe mit einer column, da steht welches objekt gemessen wurde
    # """
    # return df_labeled

df_new = transform_data(df)

# print(df)
print(df_new)

         0         1         2         3         4         5         6   \
0  5.051546  5.051546  5.051546  5.051546  6.030928  6.030928  6.030928   

         7         8         9   ...        51        52        53        54  \
0  6.030928  5.704467  5.704467  ...  5.034364  5.051546  5.051546  5.051546   

         55        56        57        58        59        60  
0  5.051546  5.051546  5.051546  5.034364  5.051546  5.034364  

[1 rows x 61 columns]
         1         2         3         4         5         6         7   \
0  5.051546  5.051546  5.051546  6.030928  6.030928  6.030928  6.030928   

         8         9         10  ...        52        53        54        55  \
0  5.704467  5.704467  5.704467  ...  5.051546  5.051546  5.051546  5.051546   

         56        57        58        59        60        61  
0  5.051546  5.051546  5.034364  5.051546  5.034364  5.051546  

[1 rows x 61 columns]
         2         3         4         5         6         7         8   \